# Image classifiers training
## 1. Data Augmentation

In [11]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms

train_dir = './train_images'
test_dir = './test_images'

# Define the transformations for augmentation
data_transforms = {
    'train': transforms.Compose([
        transforms.Grayscale(), 
        transforms.RandomResizedCrop(36),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0,),std=(1,))
    ]),
    'test':transforms.Compose([
        transforms.Grayscale(),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0,),std=(1,))
    ])        
}
train_data = torchvision.datasets.ImageFolder(train_dir, transform=data_transforms['train'])
test_data = torchvision.datasets.ImageFolder(test_dir, transform=data_transforms['test'])

train_data_size = len(train_data)
test_data_size = len(test_data)

In [12]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
from torch.utils.data.sampler import SubsetRandomSampler

valid_size = 0.2
batch_size = 32
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

num_train = len(train_data)
indices_train = list(range(num_train))
np.random.shuffle(indices_train)
split_tv = int(np.floor(valid_size * num_train))
train_new_idx, valid_idx = indices_train[split_tv:],indices_train[:split_tv]

train_sampler = SubsetRandomSampler(train_new_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, num_workers=5)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler, num_workers=5)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=5)
classes = ('noface','face')

In [14]:
import torch.nn as nn
import torch.nn.functional as F


class Net_3_layers(nn.Module):
    def __init__(self):
        super(Net_3_layers, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 6 * 6, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net_3_layers().to(device)

In [15]:
class Net_4_layers(nn.Module):
    def __init__(self):
        super(Net_4_layers, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 6 * 6, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16) # Add a new fully connected layer
        self.fc4 = nn.Linear(16, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model_1 = Net_4_layers().to(device)

In [16]:
def train_NN(n_epochs, optimizer, model, criterion, early_stop_value):
    EPOCHS_WAIT_BEFORE_EARLY_STOPPING = 5 # tolerate some epoch with no improvement to avoid local minimum
    epochs_count = 0

    for epoch in range(1, n_epochs+1):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        
        loss_average = running_loss / len(train_loader)
        print(f'[{epoch}, {i + 1:5d}] loss: {loss_average:.3f}')
        
        # stop the training if the loss is lower than the threshold
        if (loss_average < early_stop_value):
            epochs_count = epochs_count + 1
            if (epochs_count == EPOCHS_WAIT_BEFORE_EARLY_STOPPING):
                break

    print('Finished Training')

In [17]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer_SGD = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer_Adam = optim.Adam(model_1.parameters(), lr=0.001, weight_decay=1e-4) # Use Adam optimization
n_epochs = 20
early_stop_value = 0.1

train_NN(n_epochs, optimizer_SGD, model, criterion, early_stop_value)
train_NN(n_epochs, optimizer_Adam, model_1, criterion, early_stop_value)

# save the model after training
path = "models/3-layer-net.pth"
torch.save(model.state_dict(), path)

path_1 = "models/4-layer-net.pth"
torch.save(model_1.state_dict(), path)

[1,  2293] loss: 0.605
[2,  2293] loss: 0.581
[3,  2293] loss: 0.476
[4,  2293] loss: 0.378
[5,  2293] loss: 0.331
[6,  2293] loss: 0.295
[7,  2293] loss: 0.274
[8,  2293] loss: 0.247
[9,  2293] loss: 0.221
[10,  2293] loss: 0.211
[11,  2293] loss: 0.193
[12,  2293] loss: 0.181
[13,  2293] loss: 0.173
[14,  2293] loss: 0.164
[15,  2293] loss: 0.158
[16,  2293] loss: 0.154
[17,  2293] loss: 0.147
[18,  2293] loss: 0.140
[19,  2293] loss: 0.136
[20,  2293] loss: 0.132
Finished Training


In [6]:
def get_classifier_accuracy(model):
    # number of no face and face images in the test dataset
    NO_FACE = 6831
    FACE = 797

    correct = 0
    correct_no_face = 0
    correct_has_face = 1
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct_no_face += ((predicted == 0) & (labels == 0)).sum().item()
            correct_has_face += ((predicted == 1) & (labels == 1)).sum().item()
            correct += (predicted == labels).sum().item()
    
    print("correct no face = {:.2f}, has face = {:.2f}".format(100 * correct_no_face / NO_FACE, 100 * correct_has_face / FACE))

    return 100 * correct / total

In [19]:
model.load_state_dict(torch.load('models/3-layer-net.pth'))
model_1.load_state_dict(torch.load('models/4-layer-net.pth'))

model_accuracy = get_classifier_accuracy(model)
model_1_accuracy = get_classifier_accuracy(model_1)
print('Accuracy of the 3-layer classifier on the {} test images: {:.2f}'.format(test_data_size, model_accuracy))
print('Accuracy of the 4-layer classifier on the {} test images: {:.2f}'.format(test_data_size, model_1_accuracy))

correct no face = 99.00, has face = 83.69
correct no face = 99.60, has face = 88.33
Accuracy of the 3-layer classifier on the 7628 test images: 97.39
Accuracy of the 4-layer classifier on the 7628 test images: 98.41


**Observation**
By applying data augmentation techniques on the fly, we obtained a classifier of 98% accuracy, which is 6% higher than the one trained with original data.
Then we added a fully-connected layer (64,32) and replaced SGD with Adam as optimization function. The accuracy of image classifier increased by 0.3%.

## 2. Transfer Learning
Since RESNET18 was trained on RGB images, we do not apply GrayScale transform in pre-processing step.

In [18]:
# Define the transformations for augmentation
data_transforms_RGB = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(36),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test':transforms.Compose([
        transforms.ToTensor(), 
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])        
}
train_data_RGB = torchvision.datasets.ImageFolder(train_dir, transform=data_transforms_RGB['train'])
test_data_RGB = torchvision.datasets.ImageFolder(test_dir, transform=data_transforms_RGB['test'])


In [19]:
train_loader = torch.utils.data.DataLoader(train_data_RGB, batch_size=batch_size, sampler=train_sampler, num_workers=5)
valid_loader = torch.utils.data.DataLoader(train_data_RGB, batch_size=batch_size, sampler=valid_sampler, num_workers=5)
test_loader = torch.utils.data.DataLoader(test_data_RGB, batch_size=batch_size, shuffle=True, num_workers=5)

In [20]:
import torch.nn as nn
model_ft = torchvision.models.resnet18(weights='IMAGENET1K_V1')
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
model_2 = model_ft.to(device)

In [21]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer_Adam = optim.Adam(model_2.parameters(), lr=0.001, weight_decay=1e-4) # Use Adam optimization
n_epochs = 20

train_NN(n_epochs, optimizer_Adam, model_2, criterion)


[2,  2000] loss: 0.107
[3,  2000] loss: 0.078
[4,  2000] loss: 0.072
[5,  2000] loss: 0.066
[6,  2000] loss: 0.058
[7,  2000] loss: 0.057
[8,  2000] loss: 0.055
[9,  2000] loss: 0.054
[10,  2000] loss: 0.052
[11,  2000] loss: 0.050
[12,  2000] loss: 0.049
[13,  2000] loss: 0.051
[14,  2000] loss: 0.049
[15,  2000] loss: 0.049
[16,  2000] loss: 0.047
[17,  2000] loss: 0.049
[18,  2000] loss: 0.046
[19,  2000] loss: 0.049
[20,  2000] loss: 0.046
[21,  2000] loss: 0.045
Finished Training


In [23]:
model_2_accuracy = get_classifier_accuracy(model_2)
print('Accuracy of the pretrained RESNET18 classifier on the 7628 test images: {:.2f}'.format(model_2_accuracy))

Accuracy of the pretrained RESNET18 classifier on the 7628 test images: 57.56
